In [ ]:
# |default_exp mongo

# DB variables

In [ ]:
# |export

from dotenv import load_dotenv, find_dotenv
from helper import load_env
import os


def get_DB_VARS():
    load_env()
    DB_HOST = os.getenv("DB_HOST")
    DB_NAME = os.getenv("DB_NAME")
    DB_LOGS = os.getenv("DB_LOGS")
    return DB_HOST, DB_NAME, DB_LOGS

# logs

In [ ]:
# |export

from connection import get_collection
from datetime import datetime
import pymongo


def mongolog(operation: str, entite: str, desc: str):
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    if DB_LOGS in ["true", "True"]:
        coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
        coll_logs.insert_one(
            {
                "operation": operation,
                "entite": entite,
                "desc": desc,
                "date": datetime.now(),
            }
        )


def print_logs(n: int = 10):
    """
    Print the last n logs
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
    for i, log in enumerate(coll_logs.find().sort("date", pymongo.DESCENDING)):
        if i == n:
            break
        print(log)

In [ ]:
print_logs(5)

{'_id': ObjectId('677a39e52e69c4944df8b0d5'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 1, 427000)}
{'_id': ObjectId('677a39e52e69c4944df8b0d2'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 1, 422000)}
{'_id': ObjectId('677a39992e69c4944df8b0ce'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 49, 45, 117000)}
{'_id': ObjectId('677a39992e69c4944df8b0cb'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 49, 45, 112000)}
{'_id': ObjectId('677a391e2e69c4944df8b0c8'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 8, 47, 42, 171000)}


# Base entity

c'est une classe de base issue de la refactorisation de Auteur et Editeur proposée par github copilot


> Étapes de Raisonnement pour la Factorisation
> Analyser les Classes Existantes :
> 
> Les classes Auteur et Editeur ont des méthodes similaires pour vérifier l'existence, insérer, mettre à jour et supprimer des documents dans la base de données MongoDB.
> Les méthodes exists, keep, remove, et get_oid sont communes aux deux classes.
> Identifier les Comportements Communs :
> 
> Les comportements communs peuvent être regroupés dans une classe de base.
> Les classes spécifiques (Auteur et Editeur) peuvent hériter de cette classe de base et ajouter des comportements spécifiques si nécessaire.
> Créer une Classe de Base :
> 
> Créer une classe de base BaseEntity qui contient les méthodes communes.
> La classe de base gérera les interactions avec la base de données.
> Hériter de la Classe de Base :
> 
> Les classes Auteur et Editeur hériteront de BaseEntity.
> Elles définiront uniquement les comportements spécifiques à chaque entité.



In [ ]:
# |export

from bson import ObjectId
from connection import get_collection


class BaseEntity:
    def __init__(self, nom: str, collection_name: str):
        """
        BaseEntity is a class that represents a generic entity in the database.
        :param nom: The name of the entity.
        :param collection_name: The name of the collection.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.nom = nom

    def exists(self) -> bool:
        """
        Check if the entity exists in the database.
        :return: True if the entity exists, False otherwise.
        """
        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self):
        """
        Keep the entity in the database.
        """
        if not self.exists():
            mongolog("insert", self.collection.name, self.nom)
            self.collection.insert_one({"nom": self.nom})
        else:
            mongolog("update", self.collection.name, self.nom)

    def remove(self):
        """
        Remove the entity from the database.
        """
        self.collection.delete_one({"nom": self.nom})
        mongolog("delete", self.collection.name, self.nom)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the entity.
        :return: The object id of the entity. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

# Auteur

In [ ]:
# |export


class Auteur(BaseEntity):
    def __init__(self, nom: str):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        """
        super().__init__(nom, "auteurs")

In [ ]:
auteur1 = Auteur("Victor Hugo")
print(f"Est-ce que auteur1 existe ? {auteur1.exists()}")

auteur1.keep()
print(f"et maintenant, st-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et voici l'id de auteur1 : {auteur1.get_oid()}")

auteur1.remove()
print(f"après nettoyage, est-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et son oid : {auteur1.get_oid()}")

Est-ce que auteur1 existe ? False
et maintenant, st-ce que auteur1 existe ? True
et voici l'id de auteur1 : 677a3a0c813731f843f0be82
après nettoyage, est-ce que auteur1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('677a3a0c813731f843f0be84'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 40, 970000)}
{'_id': ObjectId('677a3a0c813731f843f0be81'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 40, 964000)}
{'_id': ObjectId('677a39e52e69c4944df8b0d5'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 1, 427000)}
{'_id': ObjectId('677a39e52e69c4944df8b0d2'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 1, 422000)}
{'_id': ObjectId('677a39992e69c4944df8b0ce'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 49, 45, 117000)}


# Editeur

In [ ]:
# |export


class Editeur(BaseEntity):
    def __init__(self, nom: str):
        """
        Editeur is a class that represents a publisher in the database editeurs.
        :param nom: The name of the publisher.
        """
        super().__init__(nom, "editeurs")

In [ ]:
editeur1 = Editeur("Gallimard")
print(f"Est-ce que editeur1 existe ? {editeur1.exists()}")

editeur1.keep()
print(f"et maintenant, st-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et voici l'id de editeur1 : {editeur1.get_oid()}")

editeur1.remove()
print(f"après nettoyage, est-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et son oid : {editeur1.get_oid()}")

Est-ce que editeur1 existe ? False
et maintenant, st-ce que editeur1 existe ? True
et voici l'id de editeur1 : 677a3a0d813731f843f0be89
après nettoyage, est-ce que editeur1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('677a3a0d813731f843f0be8b'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 15000)}
{'_id': ObjectId('677a3a0d813731f843f0be88'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 6000)}
{'_id': ObjectId('677a3a0c813731f843f0be84'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 40, 970000)}
{'_id': ObjectId('677a3a0c813731f843f0be81'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 40, 964000)}
{'_id': ObjectId('677a39e52e69c4944df8b0d5'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 1, 427000)}


# Critique

In [ ]:
# |export


class Critique(BaseEntity):
    def __init__(self, nom: str):
        """
        Critique is a class that represents a "critique" in the database critiques.
        :param nom: The name of the "critique".
        """
        super().__init__(nom, "critiques")

In [ ]:
critique1 = Critique("Patricia Martin")
print(f"Est-ce que critique1 existe ? {critique1.exists()}")

critique1.keep()
print(f"et maintenant, st-ce que critique1 existe ? {critique1.exists()}")
print(f"et voici l'id de critique1 : {critique1.get_oid()}")

critique1.remove()
print(f"après nettoyage, est-ce que critique1 existe ? {critique1.exists()}")
print(f"et son oid : {critique1.get_oid()}")

Est-ce que critique1 existe ? False
et maintenant, st-ce que critique1 existe ? True
et voici l'id de critique1 : 677a3a0d813731f843f0be90
après nettoyage, est-ce que critique1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('677a3a0d813731f843f0be92'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 58000)}
{'_id': ObjectId('677a3a0d813731f843f0be8f'), 'operation': 'insert', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 53000)}
{'_id': ObjectId('677a3a0d813731f843f0be8b'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 15000)}
{'_id': ObjectId('677a3a0d813731f843f0be88'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 8, 51, 41, 6000)}
{'_id': ObjectId('677a3a0c813731f843f0be84'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 8, 51, 40, 970000)}


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("mongo helper.ipynb", ".")